In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules

%matplotlib inline

print("OK\n")

OK



In [2]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.loc[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.loc[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

In [3]:
app_train_origin =  pd.read_csv('./home-credit-default-risk/exports/app_train_gbm.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test_gbm.csv')
app_train_origin.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,0,0,0,-0.577538,0.142129,-0.478095,-0.166152,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-0.513331
1,100003,0,0,1,0,1,-0.577538,0.426792,1.725450,0.592666,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589
2,100004,0,1,0,1,0,-0.577538,-0.427196,-1.152888,-1.404665,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589
3,100006,0,0,1,0,0,-0.577538,-0.142533,-0.711430,0.177862,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.001001,-0.000539,0.034180,-0.029661,0.003934,0.104634
4,100007,0,0,0,0,0,-0.577538,-0.199466,-0.213734,-0.361756,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589


In [4]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,0,0,0,-0.577538,0.142129,-0.478095,-0.166152,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-0.513331
1,100003,0,0,1,0,1,-0.577538,0.426792,1.725450,0.592666,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589
2,100004,0,1,0,1,0,-0.577538,-0.427196,-1.152888,-1.404665,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589
3,100006,0,0,1,0,0,-0.577538,-0.142533,-0.711430,0.177862,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.001001,-0.000539,0.034180,-0.029661,0.003934,0.104634
4,100007,0,0,0,0,0,-0.577538,-0.199466,-0.213734,-0.361756,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589


In [5]:
## 特徴量の作成

In [7]:
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()
app_train_domain.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,0,0,0,-0.577538,0.142129,-0.478095,-0.166152,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-0.513331
1,100003,0,0,1,0,1,-0.577538,0.426792,1.725450,0.592666,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589
2,100004,0,1,0,1,0,-0.577538,-0.427196,-1.152888,-1.404665,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589
3,100006,0,0,1,0,0,-0.577538,-0.142533,-0.711430,0.177862,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.001001,-0.000539,0.034180,-0.029661,0.003934,0.104634
4,100007,0,0,0,0,0,-0.577538,-0.199466,-0.213734,-0.361756,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589


In [293]:
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']
app_train_domain.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT
0,100002,1,0,0,0,0,-0.577538,0.142129,-0.478095,-0.166152,...,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-0.513331,-3.363804,-1.169020,0.347529,-0.302755
1,100003,0,0,1,0,1,-0.577538,0.426792,1.725450,0.592666,...,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589,4.042836,1.388654,0.343485,2.758134
2,100004,0,1,0,1,0,-0.577538,-0.427196,-1.152888,-1.404665,...,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589,2.698732,3.288103,1.218388,0.657423
3,100006,0,0,1,0,0,-0.577538,-0.142533,-0.711430,0.177862,...,-0.001001,-0.000539,0.034180,-0.029661,0.003934,0.104634,4.991320,-1.247863,-0.250007,0.695791
4,100007,0,0,0,0,0,-0.577538,-0.199466,-0.213734,-0.361756,...,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589,1.071533,1.813620,1.692547,0.530187


In [294]:
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']
app_test_domain.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT
0,100001,0,1,0,0,-0.577538,-0.142533,-0.075097,-0.451795,-0.238617,...,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,-1.086589,0.526876,3.169750,6.016117,0.637696
1,100005,0,0,0,0,-0.577538,-0.294354,-0.934825,-0.671927,-0.969511,...,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,0.633186,3.175860,2.282721,0.718773,1.040592
2,100013,0,0,1,0,-0.577538,0.142129,0.159601,2.943946,0.248645,...,-0.082511,-0.067550,-0.180649,-0.310034,0.995174,1.206445,1.122932,20.713163,18.445614,0.527104
3,100028,0,1,0,0,2.192084,0.616567,2.424840,1.511693,2.806772,...,-0.082511,-0.067550,-0.180649,-0.310034,-0.358577,0.633186,3.932808,2.451790,0.623420,-0.984417
4,100038,0,0,1,1,0.807273,0.047242,0.065776,0.342107,0.236464,...,-0.022074,-0.032718,-0.146122,-0.299577,0.460191,-0.054314,1.392319,7.241636,5.201131,-0.680321


### CREDIT_INCOME_PERCENT

In [295]:
app_train_domain['CREDIT_INCOME_PERCENT'].isnull().sum()

0

In [297]:
app_train_domain[['TARGET', 'CREDIT_INCOME_PERCENT']].corr()

,TARGET,CREDIT_INCOME_PERCENT
TARGET,1.000000,-0.002194
CREDIT_INCOME_PERCENT,-0.002194,1.000000


### ANNUITY_INCOME_PERCENT

In [298]:
app_train_domain['ANNUITY_INCOME_PERCENT'].isnull().sum()

0

In [299]:
app_train_domain[['TARGET', 'ANNUITY_INCOME_PERCENT']].corr()

,TARGET,ANNUITY_INCOME_PERCENT
TARGET,1.000000,-0.000006
ANNUITY_INCOME_PERCENT,-0.000006,1.000000


### CREDIT_TERM

In [300]:
app_train_domain['CREDIT_TERM'].isnull().sum()

0

In [301]:
app_train_domain[['TARGET', 'CREDIT_TERM']].corr()

,TARGET,CREDIT_TERM
TARGET,1.000000,-0.003646
CREDIT_TERM,-0.003646,1.000000


### DAYS_EMPLOYED_PERCENT

In [302]:
app_train_domain['DAYS_EMPLOYED_PERCENT'].isnull().sum()

0

In [303]:
app_train_domain[['TARGET', 'DAYS_EMPLOYED_PERCENT']].corr()

,TARGET,DAYS_EMPLOYED_PERCENT
TARGET,1.000000,0.001044
DAYS_EMPLOYED_PERCENT,0.001044,1.000000


## 特徴量の選択

In [304]:
category_columns = []

### NAME_CONTRACT_TYPE

In [305]:
app_train['NAME_CONTRACT_TYPE'].value_counts()

0    278232
1     29279
Name: NAME_CONTRACT_TYPE, dtype: int64

In [306]:
app_train['NAME_CONTRACT_TYPE'].isnull().sum()

0

### CODE_GENDER

In [20]:
app_train['CODE_GENDER'].isnull().sum()

0

In [21]:
app_train['CODE_GENDER'].value_counts()

1    202448
0    105059
2         4
Name: CODE_GENDER, dtype: int64

### FLAG_OWN_CAR

In [22]:
app_train['FLAG_OWN_CAR'].isnull().sum()

0

In [23]:
app_train['FLAG_OWN_CAR'].value_counts()

0    202924
1    104587
Name: FLAG_OWN_CAR, dtype: int64

### FLAG_OWN_REALTY

In [24]:
app_train['FLAG_OWN_REALTY'].isnull().sum()

0

In [25]:
app_train['FLAG_OWN_REALTY'].value_counts()

0    213312
1     94199
Name: FLAG_OWN_REALTY, dtype: int64

### CNT_CHILDREN

In [26]:
app_train['CNT_CHILDREN'].isnull().sum()

0

In [27]:
app_train['CNT_CHILDREN'].value_counts()

-0.577538     215371
 0.807273      61119
 2.192084      26749
 3.576895       3717
 4.961706        429
 6.346517         84
 7.731328         21
 9.116139          7
 18.809816         3
 10.500950         2
 11.885761         2
 16.040194         2
 13.270572         2
 25.733871         2
 14.655383         1
Name: CNT_CHILDREN, dtype: int64

### AMT_INCOME_TOTAL

In [28]:
app_train['AMT_INCOME_TOTAL'].isnull().sum()

0

### AMT_CREDIT

In [29]:
app_train['AMT_CREDIT'].isnull().sum()

0

### AMT_ANNUITY

In [30]:
app_train['AMT_ANNUITY'].isnull().sum()

0

### AMT_GOODS_PRICE

In [31]:
app_train['AMT_GOODS_PRICE'].isnull().sum()

0

### NAME_TYPE_SUITE

In [32]:
app_train['NAME_TYPE_SUITE'].isnull().sum()

0

In [33]:
app_train['NAME_TYPE_SUITE'].value_counts()

0    248526
1     40149
2     11370
3      3267
6      1770
5      1292
4       866
7       271
Name: NAME_TYPE_SUITE, dtype: int64

In [34]:
app_train['NAME_TYPE_SUITE'].fillna('BLANK', inplace=True)
app_test['NAME_TYPE_SUITE'].fillna('BLANK', inplace=True)

In [35]:
app_train['NAME_TYPE_SUITE'].isnull().sum()

0

### NAME_INCOME_TYPE

In [36]:
app_train['NAME_INCOME_TYPE'].isnull().sum()

0

In [37]:
app_train['NAME_INCOME_TYPE'].value_counts()

0    158774
2     71617
3     55362
1     21703
4        22
5        18
6        10
7         5
Name: NAME_INCOME_TYPE, dtype: int64

### NAME_EDUCATION_TYPE

In [38]:
app_train['NAME_EDUCATION_TYPE'].isnull().sum()

0

In [39]:
app_train['NAME_EDUCATION_TYPE'].value_counts()

0    218391
1     74863
2     10277
3      3816
4       164
Name: NAME_EDUCATION_TYPE, dtype: int64

In [40]:
### NAME_FAMILY_STATUS

In [41]:
app_train['NAME_FAMILY_STATUS'].isnull().sum()

0

In [42]:
app_train['NAME_FAMILY_STATUS'].value_counts()

1    196432
0     45444
2     29775
4     19770
3     16088
5         2
Name: NAME_FAMILY_STATUS, dtype: int64

In [43]:
### NAME_HOUSING_TYPE

In [44]:
app_train['NAME_HOUSING_TYPE'].isnull().sum()

0

In [45]:
app_train['NAME_HOUSING_TYPE'].value_counts()

0    272868
2     14840
3     11183
1      4881
4      2617
5      1122
Name: NAME_HOUSING_TYPE, dtype: int64

In [46]:
### REGION_POPULATION_RELATIVE

In [47]:
app_train['REGION_POPULATION_RELATIVE'].isnull().sum()

0

In [48]:
app_train['REGION_POPULATION_RELATIVE'].value_counts()

 1.078997    16408
 1.832942    13442
 0.714822    12163
 0.310593    11950
 0.399377    11601
             ...  
-1.406316      467
-1.412389      235
-1.470229       39
-1.440947       28
-1.487798        2
Name: REGION_POPULATION_RELATIVE, Length: 81, dtype: int64

### DAYS_BIRTH

In [49]:
app_train['DAYS_BIRTH'].isnull().sum()

0

In [50]:
app_train['DAYS_BIRTH'].value_counts()

 0.524291    43
 0.585703    42
-0.506649    41
 1.378786    41
 0.060952    40
             ..
-2.030486     1
-2.023612     1
 1.837770     1
-2.076316     1
-2.067838     1
Name: DAYS_BIRTH, Length: 17460, dtype: int64

### DAYS_EMPLOYED

In [51]:
app_train['DAYS_EMPLOYED'].isnull().sum()

0

In [52]:
app_train['DAYS_EMPLOYED'].value_counts()

 2.133617    55374
-0.453122      156
-0.453292      152
-0.453334      151
-0.453115      151
             ...  
-0.550527        1
-0.535422        1
-0.523736        1
-0.518661        1
-0.513246        1
Name: DAYS_EMPLOYED, Length: 12574, dtype: int64

### DAYS_REGISTRATION

In [53]:
app_train['DAYS_REGISTRATION'].isnull().sum()

0

In [54]:
app_train['DAYS_REGISTRATION'].value_counts()

 1.415069    113
 1.413366     98
 1.413650     96
 1.414218     92
 1.414785     92
            ... 
-3.007448      1
-2.851326      1
-2.786890      1
-2.844797      1
-2.785187      1
Name: DAYS_REGISTRATION, Length: 15688, dtype: int64

### DAYS_ID_PUBLISH

In [55]:
app_train['DAYS_ID_PUBLISH'].isnull().sum()

0

In [56]:
app_train['DAYS_ID_PUBLISH'].value_counts()

-0.701447    169
-0.729272    162
-0.696809    161
-0.942595    159
-0.835933    158
            ... 
-1.922423      1
-1.938985      1
-1.827686      1
-1.964823      1
-2.131109      1
Name: DAYS_ID_PUBLISH, Length: 6168, dtype: int64

In [57]:
### OWN_CAR_AGE

In [58]:
app_train['OWN_CAR_AGE'].isnull().sum()

0

In [59]:
app_train['OWN_CAR_AGE'].value_counts()

-0.754782    7424
-0.888522    6382
-1.289742    6370
-0.621042    5887
-1.423482    5852
             ... 
 0.030064       1
 0.684113       1
 0.876012       1
 0.343969       1
-0.947796       1
Name: OWN_CAR_AGE, Length: 202991, dtype: int64

### FLAG_MOBIL

In [60]:
app_train['FLAG_MOBIL'].isnull().sum()

0

In [61]:
app_train['FLAG_MOBIL'].value_counts()

 0.001803      307510
-554.535842         1
Name: FLAG_MOBIL, dtype: int64

### FLAG_EMP_PHONE

In [62]:
app_train['FLAG_EMP_PHONE'].isnull().sum()

0

In [63]:
app_train['FLAG_EMP_PHONE'].value_counts()

 0.468697    252125
-2.133575     55386
Name: FLAG_EMP_PHONE, dtype: int64

### FLAG_WORK_PHONE

In [64]:
app_train['FLAG_WORK_PHONE'].isnull().sum()

0

In [65]:
app_train['FLAG_WORK_PHONE'].value_counts()

-0.499013    246203
 2.003956     61308
Name: FLAG_WORK_PHONE, dtype: int64

### FLAG_CONT_MOBILE

In [66]:
app_train['FLAG_CONT_MOBILE'].isnull().sum()

0

In [67]:
app_train['FLAG_CONT_MOBILE'].value_counts()

 0.043245     306937
-23.124304       574
Name: FLAG_CONT_MOBILE, dtype: int64

### FLAG_PHONE

In [68]:
app_train['FLAG_PHONE'].isnull().sum()

0

In [69]:
app_train['FLAG_PHONE'].value_counts()

-0.625259    221080
 1.599337     86431
Name: FLAG_PHONE, dtype: int64

### FLAG_EMAIL

In [70]:
app_train['FLAG_EMAIL'].isnull().sum()

0

In [71]:
app_train['FLAG_EMAIL'].value_counts()

-0.245215    290069
 4.078050     17442
Name: FLAG_EMAIL, dtype: int64

In [72]:
### OCCUPATION_TYPE

In [73]:
app_train['OCCUPATION_TYPE'].isnull().sum()

0

In [74]:
app_train['OCCUPATION_TYPE'].value_counts()

4     96391
0     55186
6     32102
1     27570
3     21371
5     18603
12    11380
2      9813
10     8537
11     6721
8      5946
7      4653
9      2652
14     2093
13     1348
16     1305
15      751
18      563
17      526
Name: OCCUPATION_TYPE, dtype: int64

### CNT_FAM_MEMBERS

In [75]:
app_train['CNT_FAM_MEMBERS'].isnull().sum()

0

In [76]:
app_train['CNT_FAM_MEMBERS'].value_counts()

-0.167636     158357
-1.265719      67847
 0.930447      52601
 2.028530      24697
 3.126613       3478
 4.224696        408
 5.322779         81
 6.420863         20
 7.518946          6
 8.617029          3
 13.009361         2
 10.813195         2
 19.597860         2
 15.205527         2
 11.911278         1
-0.439899          1
 14.107444         1
-0.569541          1
 9.715112          1
Name: CNT_FAM_MEMBERS, dtype: int64

### REGION_RATING_CLIENT

In [77]:
app_train['REGION_RATING_CLIENT'].isnull().sum()

0

In [78]:
app_train['REGION_RATING_CLIENT'].value_counts()

-0.103064    226984
 1.861445     48330
-2.067573     32197
Name: REGION_RATING_CLIENT, dtype: int64

### REGION_RATING_CLIENT_W_CITY

In [79]:
app_train['REGION_RATING_CLIENT_W_CITY'].isnull().sum()

0

In [80]:
app_train['REGION_RATING_CLIENT_W_CITY'].value_counts()

-0.062699    229484
 1.926416     43860
-2.051813     34167
Name: REGION_RATING_CLIENT_W_CITY, dtype: int64

### WEEKDAY_APPR_PROCESS_START

In [81]:
app_train['WEEKDAY_APPR_PROCESS_START'].isnull().sum()

0

In [82]:
app_train['WEEKDAY_APPR_PROCESS_START'].value_counts()

6    53901
0    51934
1    50714
2    50591
5    50338
4    33852
3    16181
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64

### HOUR_APPR_PROCESS_START

In [83]:
app_train['HOUR_APPR_PROCESS_START'].isnull().sum()

0

In [84]:
app_train['HOUR_APPR_PROCESS_START'].value_counts()

-0.631821    37722
-0.325620    37229
-0.019419    34233
 0.286782    30959
 0.592983    27682
-0.938022    27384
 0.899185    24839
 1.205386    20385
-1.244224    15127
 1.511587    14900
-1.550425     9131
 1.817788     9049
-1.856626     5842
 2.123989     3848
-2.162827     3638
-2.469028     2090
-2.775229     1230
 2.430190     1196
 2.736392      405
-3.081431      305
 3.042593      150
-3.387632       86
 3.348794       41
-3.693833       40
Name: HOUR_APPR_PROCESS_START, dtype: int64

### REG_REGION_NOT_LIVE_REGION

In [85]:
app_train['REG_REGION_NOT_LIVE_REGION'].isnull().sum()

0

In [86]:
app_train['REG_REGION_NOT_LIVE_REGION'].value_counts()

-0.124004    302854
 8.064242      4657
Name: REG_REGION_NOT_LIVE_REGION, dtype: int64

### REG_REGION_NOT_WORK_REGION

In [87]:
app_train['REG_REGION_NOT_WORK_REGION'].isnull().sum()

0

In [88]:
app_train['REG_REGION_NOT_WORK_REGION'].value_counts()

-0.231267    291899
 4.324013     15612
Name: REG_REGION_NOT_WORK_REGION, dtype: int64

### LIVE_REGION_NOT_WORK_REGION

In [89]:
app_train['LIVE_REGION_NOT_WORK_REGION'].isnull().sum()

0

In [90]:
app_train['LIVE_REGION_NOT_WORK_REGION'].value_counts()

-0.205869    295008
 4.857466     12503
Name: LIVE_REGION_NOT_WORK_REGION, dtype: int64

### REG_CITY_NOT_LIVE_CITY

In [91]:
app_train['REG_CITY_NOT_LIVE_CITY'].isnull().sum()

0

In [92]:
app_train['REG_CITY_NOT_LIVE_CITY'].value_counts()

-0.291208    283472
 3.433973     24039
Name: REG_CITY_NOT_LIVE_CITY, dtype: int64

### REG_CITY_NOT_WORK_CITY

In [93]:
app_train['REG_CITY_NOT_WORK_CITY'].isnull().sum()

0

In [94]:
app_train['REG_CITY_NOT_WORK_CITY'].value_counts()

-0.547236    236644
 1.827367     70867
Name: REG_CITY_NOT_WORK_CITY, dtype: int64

### LIVE_CITY_NOT_WORK_CITY

In [95]:
app_train['LIVE_CITY_NOT_WORK_CITY'].isnull().sum()

0

In [96]:
app_train['LIVE_CITY_NOT_WORK_CITY'].value_counts()

-0.467814    252296
 2.137601     55215
Name: LIVE_CITY_NOT_WORK_CITY, dtype: int64

### ORGANIZATION_TYPE

In [97]:
app_train['ORGANIZATION_TYPE'].isnull().sum()

0

In [98]:
app_train['ORGANIZATION_TYPE'].value_counts()

0     67992
5     55374
9     38412
4     16683
7     11193
8     10553
2     10404
1      8893
14     7831
13     6880
11     6721
27     5984
19     5398
35     3492
38     3368
36     3278
22     3247
12     2958
15     2704
16     2634
37     2507
30     2454
26     2341
10     2204
28     2157
18     1974
23     1900
31     1811
17     1575
24     1327
34     1307
25     1187
20     1039
33      966
6       950
29      877
40      631
46      599
39      597
54      577
21      560
41      458
49      429
56      396
32      379
43      369
45      348
44      317
48      305
51      260
42      201
57      112
47      109
3        85
52       67
53       64
50       49
55       24
Name: ORGANIZATION_TYPE, dtype: int64

### EXT_SOURCE_1

In [99]:
app_train['EXT_SOURCE_1'].isnull().sum()

0

In [100]:
app_train['EXT_SOURCE_1'].value_counts()

 0.045031    5
 0.334226    5
 0.719158    5
 0.237236    5
-1.011360    5
            ..
-0.313321    1
 2.077099    1
 0.902339    1
 0.448383    1
 1.089916    1
Name: EXT_SOURCE_1, Length: 287962, dtype: int64

### EXT_SOURCE_2

In [101]:
app_train['EXT_SOURCE_2'].isnull().sum()

0

### EXT_SOURCE_3

In [102]:
app_train['EXT_SOURCE_3'].isnull().sum()

0

### APARTMENTS_AVG

In [103]:
app_train['APARTMENTS_AVG'].isnull().sum()

0

In [104]:
app_train['APARTMENTS_AVG'].value_counts()

-0.458390    6663
-0.729574    6332
-0.322798    4404
-0.593982    3986
-1.436496    3507
             ... 
 0.000172       1
-0.004228       1
-0.001335       1
-0.006014       1
 0.000653       1
Name: APARTMENTS_AVG, Length: 158400, dtype: int64

### BASEMENTAREA_AVG

In [105]:
app_train['APARTMENTS_AVG'].isnull().sum()

0

In [106]:
app_train['APARTMENTS_AVG'].value_counts()

-0.458390    6663
-0.729574    6332
-0.322798    4404
-0.593982    3986
-1.436496    3507
             ... 
 0.000172       1
-0.004228       1
-0.001335       1
-0.006014       1
 0.000653       1
Name: APARTMENTS_AVG, Length: 158400, dtype: int64

### YEARS_BEGINEXPLUATATION_AVG

In [107]:
app_train['YEARS_BEGINEXPLUATATION_AVG'].isnull().sum()

0

In [108]:
app_train['YEARS_BEGINEXPLUATATION_AVG'].value_counts()

 0.221332    4311
 0.185942    4189
 0.197739    4171
 0.056178    4123
 0.209535    4114
             ... 
-0.007014       1
 0.001326       1
 0.000418       1
 0.000618       1
 0.001293       1
Name: YEARS_BEGINEXPLUATATION_AVG, Length: 150292, dtype: int64

### YEARS_BUILD_AVG

In [109]:
app_train['YEARS_BUILD_AVG'].isnull().sum()

0

In [110]:
app_train['YEARS_BUILD_AVG'].value_counts()

 1.080849    2999
 0.978380    2864
 0.773442    2848
-0.353717    2802
-0.251248    2761
             ... 
-0.000415       1
 0.000670       1
 0.037216       1
-0.000626       1
 0.039294       1
Name: YEARS_BUILD_AVG, Length: 204637, dtype: int64

### COMMONAREA_AVG

In [111]:
app_train['COMMONAREA_AVG'].isnull().sum()

0

In [112]:
app_train['COMMONAREA_AVG'].value_counts()

-1.065809    8442
-0.876541     544
-0.878937     475
-0.874145     446
-0.881333     414
             ... 
 0.007187       1
-0.000051       1
 0.003324       1
 0.000309       1
-0.016592       1
Name: COMMONAREA_AVG, Length: 218046, dtype: int64

### ELEVATORS_AVG

In [113]:
app_train['ELEVATORS_AVG'].isnull().sum()

0

In [114]:
app_train['ELEVATORS_AVG'].value_counts()

-0.853739    85718
 0.016056     9886
 0.885851     8806
 1.755645     6071
 0.450953     5593
             ...  
-0.013404        1
-0.005295        1
 0.016661        1
 0.000012        1
-0.053426        1
Name: ELEVATORS_AVG, Length: 164148, dtype: int64

### ENTRANCES_AVG

In [115]:
app_train['ENTRANCES_AVG'].isnull().sum()

0

In [116]:
app_train['ENTRANCES_AVG'].value_counts()

-0.166704    34007
-1.144029    22956
-0.656076    19533
 0.812039    19062
-1.633400    15380
             ...  
 0.003610        1
 0.003513        1
 0.000927        1
 0.000705        1
 0.004384        1
Name: ENTRANCES_AVG, Length: 155113, dtype: int64

### FLOORSMAX_AVG

In [117]:
app_train['FLOORSMAX_AVG'].isnull().sum()

0

In [118]:
app_train['FLOORSMAX_AVG'].value_counts()

-0.578598    61875
 1.046408    31909
-1.797840    14600
 1.453147     7926
-0.985337     6974
             ...  
-0.012646        1
-0.019550        1
 0.004108        1
-0.009528        1
 0.000765        1
Name: FLOORSMAX_AVG, Length: 153423, dtype: int64

### FLOORSMIN_AVG

In [119]:
app_train['FLOORSMIN_AVG'].isnull().sum()

0

In [120]:
app_train['FLOORSMIN_AVG'].value_counts()

-0.253134    32875
 1.568064    17845
-2.073239    17776
-1.618759     5086
 2.023636     3961
             ...  
-0.004342        1
-0.000272        1
-0.000884        1
 0.011303        1
 0.028979        1
Name: FLOORSMIN_AVG, Length: 208947, dtype: int64

### LANDAREA_AVG

In [121]:
app_train['LANDAREA_AVG'].isnull().sum()

0

In [122]:
app_train['LANDAREA_AVG'].value_counts()

-1.280547    15600
-0.061118      189
-0.669866      187
-0.366458      186
-0.944286      180
             ...  
 0.003071        1
-0.002699        1
 0.008981        1
 0.005914        1
-0.018472        1
Name: LANDAREA_AVG, Length: 186117, dtype: int64

### LIVINGAPARTMENTS_AVG

In [123]:
app_train['LIVINGAPARTMENTS_AVG'].isnull().sum()

0

In [124]:
app_train['LIVINGAPARTMENTS_AVG'].value_counts()

-0.957120    4272
-0.634968    4231
-0.473891    2778
-0.796044    2586
-0.310898    1864
             ... 
 0.155735       1
-0.000406       1
-0.002792       1
-0.004967       1
-0.040263       1
Name: LIVINGAPARTMENTS_AVG, Length: 212067, dtype: int64

In [125]:
### LIVINGAREA_AVG

In [126]:
app_train['LIVINGAREA_AVG'].isnull().sum()

0

In [127]:
app_train['LIVINGAREA_AVG'].value_counts()

-1.374341     284
-0.718221     243
-0.474739     223
-0.720784     223
-0.722065     221
             ... 
 0.000378       1
 0.002417       1
 0.040463       1
 0.000486       1
 10.516561      1
Name: LIVINGAREA_AVG, Length: 159549, dtype: int64

### NONLIVINGAPARTMENTS_AVG

In [128]:
app_train['NONLIVINGAPARTMENTS_AVG'].isnull().sum()

0

In [129]:
app_train['NONLIVINGAPARTMENTS_AVG'].value_counts()

-0.332963    54549
-0.185194    13606
-0.041215     6351
 0.106554     3714
 0.250533     2533
             ...  
 0.003631        1
-0.001348        1
 0.017238        1
-0.009232        1
-0.000430        1
Name: NONLIVINGAPARTMENTS_AVG, Length: 213900, dtype: int64

### NONLIVINGAREA_AVG

In [130]:
app_train['NONLIVINGAREA_AVG'].isnull().sum()

0

In [131]:
app_train['NONLIVINGAREA_AVG'].value_counts()

-0.605432    58735
-0.579652      546
-0.510906      454
-0.558169      440
-0.538834      415
             ...  
 0.014481        1
 0.000375        1
 0.000959        1
-0.004459        1
-0.023378        1
Name: NONLIVINGAREA_AVG, Length: 172972, dtype: int64

### APARTMENTS_MODE

In [132]:
app_train['APARTMENTS_MODE'].isnull().sum()

0

In [133]:
app_train['APARTMENTS_MODE'].value_counts()

-0.401022    7522
-0.678230    7451
-0.262418    4757
-0.539626    4388
-1.398970    3970
             ... 
-0.001671       1
 0.003412       1
-0.001820       1
 0.009585       1
-0.001978       1
Name: APARTMENTS_MODE, Length: 156821, dtype: int64

### BASEMENTAREA_MODE

In [134]:
app_train['BASEMENTAREA_MODE'].isnull().sum()

0

In [135]:
app_train['BASEMENTAREA_MODE'].value_counts()

-1.617026    16598
-0.575430      269
-0.054632      265
-0.435569      253
-0.089597      251
             ...  
-0.012081        1
-0.002816        1
 0.000293        1
 0.014241        1
-0.013815        1
Name: BASEMENTAREA_MODE, Length: 183784, dtype: int64

### YEARS_BEGINEXPLUATATION_MODE

In [136]:
app_train['YEARS_BEGINEXPLUATATION_MODE'].isnull().sum()

0

In [137]:
app_train['YEARS_BEGINEXPLUATATION_MODE'].value_counts()

 0.217127    4291
 0.206309    4173
 0.195490    4167
 0.065664    4110
 0.076482    4083
             ... 
 0.004945       1
-0.004292       1
 0.009818       1
-0.001316       1
-0.002509       1
Name: YEARS_BEGINEXPLUATATION_MODE, Length: 150228, dtype: int64

### YEARS_BUILD_MODE

In [138]:
app_train['YEARS_BUILD_MODE'].isnull().sum()

0

In [139]:
app_train['YEARS_BUILD_MODE'].value_counts()

 1.075055    2960
 0.974180    2879
-0.343402    2789
 0.873305    2763
 0.770878    2755
             ... 
-0.006110       1
 0.028678       1
-0.011171       1
 0.012239       1
-0.008078       1
Name: YEARS_BUILD_MODE, Length: 204642, dtype: int64

### COMMONAREA_MODE

In [140]:
app_train['COMMONAREA_MODE'].isnull().sum()

0

In [141]:
app_train['COMMONAREA_MODE'].value_counts()

-1.050944    9690
-0.855439     546
-0.857883     543
-0.860326     518
-0.852995     416
             ... 
 0.056662       1
-0.035278       1
 0.147343       1
 0.003227       1
 0.033500       1
Name: COMMONAREA_MODE, Length: 217993, dtype: int64

### ELEVATORS_MODE

In [142]:
app_train['ELEVATORS_MODE'].isnull().sum()

0

In [143]:
app_train['ELEVATORS_MODE'].value_counts()

-0.828159    89498
 0.063506    11629
 0.954065     9675
 1.845730     6379
 0.508232     5734
             ...  
-0.001584        1
 0.008568        1
-0.009578        1
-0.011372        1
 0.011993        1
Name: ELEVATORS_MODE, Length: 163917, dtype: int64

### ENTRANCES_MODE

In [144]:
app_train['ENTRANCES_MODE'].isnull().sum()

0

In [145]:
app_train['ENTRANCES_MODE'].value_counts()

-0.104454    36041
-1.072780    26704
-0.589319    20533
 0.865278    19678
-1.557645    19428
             ...  
 0.003774        1
 0.004649        1
 0.013625        1
-0.003040        1
-0.008032        1
Name: ENTRANCES_MODE, Length: 154858, dtype: int64

### FLOORSMAX_MODE

In [146]:
app_train['FLOORSMAX_MODE'].isnull().sum()

0

In [147]:
app_train['FLOORSMAX_MODE'].value_counts()

-0.546934    65550
 1.088615    34373
-1.774087    15616
 1.497994     8321
-0.956312     7267
             ...  
-0.007302        1
 0.000294        1
 0.004274        1
 0.001291        1
-0.003251        1
Name: FLOORSMAX_MODE, Length: 153045, dtype: int64

### FLOORSMIN_MODE

In [148]:
app_train['FLOORSMIN_MODE'].isnull().sum()

0

In [149]:
app_train['FLOORSMIN_MODE'].value_counts()

-0.227608    34403
 1.593749    19042
-2.047872    18917
-1.593352     5349
 2.049362     4105
             ...  
-0.002519        1
 0.057817        1
 0.012575        1
 0.004993        1
 0.001322        1
Name: FLOORSMIN_MODE, Length: 208667, dtype: int64

### LANDAREA_MODE

In [150]:
app_train['LANDAREA_MODE'].isnull().sum()

0

In [151]:
app_train['LANDAREA_MODE'].value_counts()

-1.250390    17453
-0.878227      208
-0.013045      193
-0.321901      192
-0.630758      188
             ...  
 0.066464        1
 0.010097        1
 0.021412        1
 0.008362        1
 0.009580        1
Name: LANDAREA_MODE, Length: 186153, dtype: int64

### LIVINGAPARTMENTS_MODE

In [152]:
app_train['LIVINGAPARTMENTS_MODE'].isnull().sum()

0

In [153]:
app_train['LIVINGAPARTMENTS_MODE'].value_counts()

-0.910338    4931
-0.578658    4797
-0.414621    2966
-0.744498    2853
-0.248781    2069
             ... 
 0.003351       1
 0.011786       1
-0.093731       1
-0.019579       1
-0.075298       1
Name: LIVINGAPARTMENTS_MODE, Length: 210935, dtype: int64

### LIVINGAREA_MODE

In [154]:
app_train['LIVINGAREA_MODE'].isnull().sum()

0

In [155]:
app_train['LIVINGAREA_MODE'].value_counts()

-1.343523     444
-0.672228     272
-0.669695     262
-0.673495     249
-0.668428     242
             ... 
-0.015031       1
-0.004621       1
-0.003534       1
-0.012465       1
 10.900647      1
Name: LIVINGAREA_MODE, Length: 159651, dtype: int64

### NONLIVINGAPARTMENTS_MODE

In [156]:
app_train['NONLIVINGAPARTMENTS_MODE'].isnull().sum()

0

In [157]:
app_train['NONLIVINGAPARTMENTS_MODE'].value_counts()

-0.316935    59255
-0.164638    14105
-0.012340     6413
 0.139957     3675
 0.292255     2492
             ...  
-0.002566        1
-0.014628        1
 0.004872        1
-0.002045        1
-0.047013        1
Name: NONLIVINGAPARTMENTS_MODE, Length: 213681, dtype: int64

### NONLIVINGAREA_MODE

In [158]:
app_train['NONLIVINGAREA_MODE'].isnull().sum()

0

In [159]:
app_train['NONLIVINGAREA_MODE'].value_counts()

-0.579055    67126
-0.555671      477
-0.481267      466
-0.508903      430
-0.553545      427
             ...  
-0.017039        1
 0.032483        1
 0.011637        1
-0.045151        1
 0.010606        1
Name: NONLIVINGAREA_MODE, Length: 173009, dtype: int64

### APARTMENTS_MEDI

In [160]:
app_train['APARTMENTS_MEDI'].isnull().sum()

0

In [161]:
app_train['APARTMENTS_MEDI'].value_counts()

-0.450337    7109
-0.722057    6687
-0.314477    4622
-0.586197    4211
-1.430096    3562
             ... 
 0.001739       1
 0.000486       1
-0.000246       1
 0.000077       1
 0.001633       1
Name: APARTMENTS_MEDI, Length: 157209, dtype: int64

### BASEMENTAREA_MEDI

In [162]:
app_train['BASEMENTAREA_MEDI'].isnull().sum()

0

In [163]:
app_train['BASEMENTAREA_MEDI'].value_counts()

-1.658730    14991
-0.113378      271
 0.402369      266
-0.629125      265
-0.285294      248
             ...  
-0.002443        1
-0.003473        1
 0.000992        1
-0.001015        1
-0.042624        1
Name: BASEMENTAREA_MEDI, Length: 183715, dtype: int64

### YEARS_BEGINEXPLUATATION_MEDI

In [164]:
app_train['YEARS_BEGINEXPLUATATION_MEDI'].isnull().sum()

0

In [165]:
app_train['YEARS_BEGINEXPLUATATION_MEDI'].value_counts()

 0.218272    4314
 0.194944    4247
 0.183280    4199
 0.206608    4138
 0.054976    4115
             ... 
-0.000216       1
-0.000145       1
-0.003621       1
 0.001884       1
 0.000998       1
Name: YEARS_BEGINEXPLUATATION_MEDI, Length: 150252, dtype: int64

### YEARS_BUILD_MEDI

In [166]:
app_train['YEARS_BUILD_MEDI'].isnull().sum()

0

In [167]:
app_train['YEARS_BUILD_MEDI'].value_counts()

 1.080693    2994
 0.977965    2883
 0.770976    2842
-0.360564    2799
 0.873704    2784
             ... 
-0.002758       1
 0.024555       1
 0.001019       1
-0.001497       1
 0.026950       1
Name: YEARS_BUILD_MEDI, Length: 204639, dtype: int64

### COMMONAREA_MEDI

In [168]:
app_train['COMMONAREA_MEDI'].isnull().sum()

0

In [169]:
app_train['COMMONAREA_MEDI'].value_counts()

-1.065457    8691
-0.876441     581
-0.874049     486
-0.878834     481
-1.031961     377
             ... 
-0.000732       1
-0.000363       1
-0.000376       1
-0.008182       1
-0.013140       1
Name: COMMONAREA_MEDI, Length: 218067, dtype: int64

### COMMONAREA_MEDI

In [170]:
app_train['COMMONAREA_MEDI'].isnull().sum()

0

In [171]:
app_train['COMMONAREA_MEDI'].value_counts()

-1.065457    8691
-0.876441     581
-0.874049     486
-0.878834     481
-1.031961     377
             ... 
-0.000732       1
-0.000363       1
-0.000376       1
-0.008182       1
-0.013140       1
Name: COMMONAREA_MEDI, Length: 218067, dtype: int64

### ELEVATORS_MEDI

In [172]:
app_train['ELEVATORS_MEDI'].isnull().sum()

0

In [173]:
app_train['ELEVATORS_MEDI'].value_counts()

-0.846902    87026
 0.023632    10832
 0.894167     9278
 1.764701     6392
 0.458899     5946
             ...  
-0.001736        1
-0.002135        1
 0.007637        1
 0.002713        1
-0.045419        1
Name: ELEVATORS_MEDI, Length: 163937, dtype: int64

### ENTRANCES_MEDI

In [174]:
app_train['ENTRANCES_MEDI'].isnull().sum()

0

In [175]:
app_train['ENTRANCES_MEDI'].value_counts()

-0.159244    35535
-1.133464    24193
-0.647061    20492
 0.816390    19750
-1.621282    16150
             ...  
 0.001735        1
-0.002165        1
-0.003930        1
-0.001645        1
 0.001127        1
Name: ENTRANCES_MEDI, Length: 154874, dtype: int64

### FLOORSMAX_MEDI

In [176]:
app_train['FLOORSMAX_MEDI'].isnull().sum()

0

In [177]:
app_train['FLOORSMAX_MEDI'].value_counts()

-0.574140    63607
 1.046113    33279
-1.789816    14832
 1.451662     8207
-0.979689     7079
             ...  
 0.008897        1
 0.004595        1
 0.001470        1
-0.001479        1
 0.000388        1
Name: FLOORSMAX_MEDI, Length: 153069, dtype: int64

### FLOORSMIN_MEDI

In [178]:
app_train['FLOORSMIN_MEDI'].isnull().sum()

0

In [179]:
app_train['FLOORSMIN_MEDI'].value_counts()

-0.251755    33737
 1.563699    18538
-2.066119    18090
-1.613073     5131
 2.017835     4068
             ...  
-0.000368        1
-0.028005        1
-0.001744        1
-0.000129        1
 0.022726        1
Name: FLOORSMIN_MEDI, Length: 208689, dtype: int64

### LANDAREA_MEDI

In [180]:
app_train['LANDAREA_MEDI'].isnull().sum()

0

In [181]:
app_train['LANDAREA_MEDI'].value_counts()

-1.281586    15919
-0.913074      197
-0.055758      194
-0.361260      186
-0.974174      180
             ...  
-0.003196        1
-0.001868        1
 0.004975        1
-0.005453        1
-0.032894        1
Name: LANDAREA_MEDI, Length: 186150, dtype: int64

### LIVINGAPARTMENTS_MEDI

In [182]:
app_train['LIVINGAPARTMENTS_MEDI'].isnull().sum()

0

In [183]:
app_train['LIVINGAPARTMENTS_MEDI'].value_counts()

-0.951649    4500
-0.627549    4497
-0.464551    2926
-0.788651    2708
-0.303449    2014
             ... 
-0.001744       1
-0.004158       1
-0.074350       1
-0.040483       1
-0.059082       1
Name: LIVINGAPARTMENTS_MEDI, Length: 211296, dtype: int64

### LIVINGAREA_MEDI

In [184]:
app_train['LIVINGAREA_MEDI'].isnull().sum()

0

In [185]:
app_train['LIVINGAREA_MEDI'].value_counts()

-1.369232     299
-0.677600     239
-0.715463     234
-0.712938     233
-0.711676     232
             ... 
-0.001497       1
-0.003467       1
 0.001055       1
 0.005339       1
 10.551334      1
Name: LIVINGAREA_MEDI, Length: 159631, dtype: int64

### NONLIVINGAPARTMENTS_MEDI

In [186]:
app_train['NONLIVINGAPARTMENTS_MEDI'].isnull().sum()

0

In [187]:
app_train['NONLIVINGAPARTMENTS_MEDI'].value_counts()

-0.329781    56097
-0.181013    14126
-0.032245     6493
 0.112708     3788
 0.261476     2557
             ...  
 0.001767        1
-0.009101        1
 0.004598        1
 0.001605        1
-0.011858        1
Name: NONLIVINGAPARTMENTS_MEDI, Length: 213728, dtype: int64

### NONLIVINGAREA_MEDI

In [188]:
app_train['NONLIVINGAREA_MEDI'].isnull().sum()

0

In [189]:
app_train['NONLIVINGAREA_MEDI'].value_counts()

-0.600059    60954
-0.574514      539
-0.553227      478
-0.521295      456
-0.506394      431
             ...  
-0.000063        1
-0.000881        1
 0.004084        1
 0.003289        1
-0.014939        1
Name: NONLIVINGAREA_MEDI, Length: 173005, dtype: int64

### FONDKAPREMONT_MODE

In [190]:
app_train['FONDKAPREMONT_MODE'].isnull().sum()

0

In [191]:
app_train['FONDKAPREMONT_MODE'].value_counts()

1    210295
0     73830
3     12080
4      5687
2      5619
Name: FONDKAPREMONT_MODE, dtype: int64

In [192]:
### HOUSETYPE_MODE

In [193]:
app_train['HOUSETYPE_MODE'].isnull().sum()

0

In [194]:
app_train['HOUSETYPE_MODE'].value_counts()

1    154297
0    150503
3      1499
2      1212
Name: HOUSETYPE_MODE, dtype: int64

### TOTALAREA_MODE

In [195]:
app_train['TOTALAREA_MODE'].isnull().sum()

0

In [196]:
app_train['TOTALAREA_MODE'].value_counts()

-1.323867    582
-0.586737    247
-0.616481    230
-0.606135    227
-0.615187    227
            ... 
-0.050656      1
-0.005495      1
-0.000116      1
 0.045986      1
 8.983027      1
Name: TOTALAREA_MODE, Length: 153547, dtype: int64

### WALLSMATERIAL_MODE

In [197]:
app_train['WALLSMATERIAL_MODE'].isnull().sum()

0

In [198]:
app_train['WALLSMATERIAL_MODE'].value_counts()

2    156341
3     66040
0     64815
1      9253
5      5362
4      2296
7      1779
6      1625
Name: WALLSMATERIAL_MODE, dtype: int64

### EMERGENCYSTATE_MODE

In [199]:
app_train['EMERGENCYSTATE_MODE'].isnull().sum()

0

In [200]:
app_train['EMERGENCYSTATE_MODE'].value_counts()

0    159428
1    145755
2      2328
Name: EMERGENCYSTATE_MODE, dtype: int64

### OBS_30_CNT_SOCIAL_CIRCLE

In [201]:
app_train['OBS_30_CNT_SOCIAL_CIRCLE'].isnull().sum()

0

In [202]:
app_train['OBS_30_CNT_SOCIAL_CIRCLE'].value_counts()

-0.593345    163910
-0.176156     48783
 0.241033     29808
 0.658222     20322
 1.075411     14143
              ...  
-0.000774         1
 0.000733         1
 0.000501         1
 0.000114         1
-0.000616         1
Name: OBS_30_CNT_SOCIAL_CIRCLE, Length: 1054, dtype: int64

### DEF_30_CNT_SOCIAL_CIRCLE

In [203]:
app_train['DEF_30_CNT_SOCIAL_CIRCLE'].isnull().sum()

0

In [204]:
app_train['DEF_30_CNT_SOCIAL_CIRCLE'].value_counts()

-0.321591    271324
 1.920782     28328
 4.163155      5323
 6.405528      1192
 8.647901       253
              ...  
-0.040890         1
 0.024667         1
-0.042425         1
-0.007389         1
 0.028872         1
Name: DEF_30_CNT_SOCIAL_CIRCLE, Length: 1031, dtype: int64

### OBS_60_CNT_SOCIAL_CIRCLE

In [205]:
app_train['OBS_60_CNT_SOCIAL_CIRCLE'].isnull().sum()

0

In [206]:
app_train['OBS_60_CNT_SOCIAL_CIRCLE'].value_counts()

-0.591490    164666
-0.170588     48870
 0.250315     29766
 0.671218     20215
 1.092121     13946
              ...  
-0.000491         1
-0.000219         1
 0.000047         1
 0.000517         1
 0.000155         1
Name: OBS_60_CNT_SOCIAL_CIRCLE, Length: 1054, dtype: int64

### DEF_60_CNT_SOCIAL_CIRCLE

In [207]:
app_train['DEF_60_CNT_SOCIAL_CIRCLE'].isnull().sum()

0

In [208]:
app_train['DEF_60_CNT_SOCIAL_CIRCLE'].value_counts()

-0.276607     280721
 2.488201      21841
 5.253008       3170
 8.017815        598
 10.782623       135
               ...  
-0.042536          1
 0.030110          1
-0.028902          1
-0.022815          1
 0.041527          1
Name: DEF_60_CNT_SOCIAL_CIRCLE, Length: 1030, dtype: int64

### DAYS_LAST_PHONE_CHANGE

In [209]:
app_train['DAYS_LAST_PHONE_CHANGE'].isnull().sum()

0

In [210]:
app_train['DAYS_LAST_PHONE_CHANGE'].value_counts()

 1.164549    37672
 1.163339     2812
 1.162130     2318
 1.160920     1763
 1.159711     1285
             ...  
-3.326223        1
-3.646733        1
-3.652781        1
-3.357669        1
-3.114565        1
Name: DAYS_LAST_PHONE_CHANGE, Length: 3774, dtype: int64

### FLAG_DOCUMENT_2

In [211]:
app_train['FLAG_DOCUMENT_2'].isnull().sum()

0

In [212]:
app_train['FLAG_DOCUMENT_2'].value_counts()

-0.006502      307498
 153.797569        13
Name: FLAG_DOCUMENT_2, dtype: int64

### FLAG_DOCUMENT_3

In [213]:
app_train['FLAG_DOCUMENT_3'].isnull().sum()

0

In [214]:
app_train['FLAG_DOCUMENT_3'].value_counts()

 0.639065    218340
-1.564786     89171
Name: FLAG_DOCUMENT_3, dtype: int64

### FLAG_DOCUMENT_4

In [215]:
app_train['FLAG_DOCUMENT_4'].isnull().sum()

0

In [216]:
app_train['FLAG_DOCUMENT_4'].value_counts()

-0.009017      307486
 110.902840        25
Name: FLAG_DOCUMENT_4, dtype: int64

### FLAG_DOCUMENT_5

In [217]:
app_train['FLAG_DOCUMENT_5'].isnull().sum()

0

In [218]:
app_train['FLAG_DOCUMENT_5'].value_counts()

-0.123882    302863
 8.072165      4648
Name: FLAG_DOCUMENT_5, dtype: int64

### FLAG_DOCUMENT_6

In [219]:
app_train['FLAG_DOCUMENT_6'].isnull().sum()

0

In [220]:
app_train['FLAG_DOCUMENT_6'].value_counts()

-0.310738    280433
 3.218150     27078
Name: FLAG_DOCUMENT_6, dtype: int64

### FLAG_DOCUMENT_7

In [221]:
app_train['FLAG_DOCUMENT_7'].isnull().sum()

0

In [222]:
app_train['FLAG_DOCUMENT_7'].value_counts()

-0.013853     307452
 72.187609        59
Name: FLAG_DOCUMENT_7, dtype: int64

### FLAG_DOCUMENT_8

In [223]:
app_train['FLAG_DOCUMENT_8'].isnull().sum()

0

In [224]:
app_train['FLAG_DOCUMENT_8'].value_counts()

-0.297632    282487
 3.359858     25024
Name: FLAG_DOCUMENT_8, dtype: int64

### FLAG_DOCUMENT_9

In [225]:
app_train['FLAG_DOCUMENT_9'].isnull().sum()

0

In [226]:
app_train['FLAG_DOCUMENT_9'].value_counts()

-0.062538     306313
 15.990215      1198
Name: FLAG_DOCUMENT_9, dtype: int64

### FLAG_DOCUMENT_10

In [227]:
app_train['FLAG_DOCUMENT_10'].isnull().sum()

0

In [228]:
app_train['FLAG_DOCUMENT_10'].value_counts()

-0.004771      307504
 209.592802         7
Name: FLAG_DOCUMENT_10, dtype: int64

### FLAG_DOCUMENT_11

In [229]:
app_train['FLAG_DOCUMENT_11'].isnull().sum()

0

In [230]:
app_train['FLAG_DOCUMENT_11'].value_counts()

-0.062669     306308
 15.956820      1203
Name: FLAG_DOCUMENT_11, dtype: int64

### FLAG_DOCUMENT_12

In [231]:
app_train['FLAG_DOCUMENT_12'].isnull().sum()

0

In [232]:
app_train['FLAG_DOCUMENT_12'].value_counts()

-0.002550      307509
 392.115417         2
Name: FLAG_DOCUMENT_12, dtype: int64

### FLAG_DOCUMENT_13

In [233]:
app_train['FLAG_DOCUMENT_13'].isnull().sum()

0

In [234]:
app_train['FLAG_DOCUMENT_13'].value_counts()

-0.059477     306427
 16.813142      1084
Name: FLAG_DOCUMENT_13, dtype: int64

### FLAG_DOCUMENT_14

In [235]:
app_train['FLAG_DOCUMENT_14'].isnull().sum()

0

In [236]:
app_train['FLAG_DOCUMENT_14'].value_counts()

-0.054269     306608
 18.426713       903
Name: FLAG_DOCUMENT_14, dtype: int64

### FLAG_DOCUMENT_15

In [237]:
app_train['FLAG_DOCUMENT_15'].isnull().sum()

0

In [238]:
app_train['FLAG_DOCUMENT_15'].value_counts()

-0.034802     307139
 28.733995       372
Name: FLAG_DOCUMENT_15, dtype: int64

### FLAG_DOCUMENT_16

In [239]:
app_train['FLAG_DOCUMENT_16'].isnull().sum()

0

In [240]:
app_train['FLAG_DOCUMENT_16'].value_counts()

-0.100138    304458
 9.986201      3053
Name: FLAG_DOCUMENT_16, dtype: int64

### FLAG_DOCUMENT_17

In [241]:
app_train['FLAG_DOCUMENT_17'].isnull().sum()

0

In [242]:
app_train['FLAG_DOCUMENT_17'].value_counts()

-0.016332     307429
 61.230173        82
Name: FLAG_DOCUMENT_17, dtype: int64

### FLAG_DOCUMENT_18

In [243]:
app_train['FLAG_DOCUMENT_18'].isnull().sum()

0

In [244]:
app_train['FLAG_DOCUMENT_18'].value_counts()

-0.090534     305011
 11.045560      2500
Name: FLAG_DOCUMENT_18, dtype: int64

### FLAG_DOCUMENT_19

In [245]:
app_train['FLAG_DOCUMENT_19'].isnull().sum()

0

In [246]:
app_train['FLAG_DOCUMENT_19'].value_counts()

-0.024402     307328
 40.980336       183
Name: FLAG_DOCUMENT_19, dtype: int64

### FLAG_DOCUMENT_20

In [247]:
app_train['FLAG_DOCUMENT_20'].isnull().sum()

0

In [248]:
app_train['FLAG_DOCUMENT_20'].value_counts()

-0.022529     307355
 44.387209       156
Name: FLAG_DOCUMENT_20, dtype: int64

### FLAG_DOCUMENT_21

In [249]:
app_train['FLAG_DOCUMENT_21'].isnull().sum()

0

In [250]:
app_train['FLAG_DOCUMENT_21'].value_counts()

-0.018305     307408
 54.630977       103
Name: FLAG_DOCUMENT_21, dtype: int64

### AMT_REQ_CREDIT_BUREAU_HOUR

In [251]:
app_train['AMT_REQ_CREDIT_BUREAU_HOUR'].isnull().sum()

0

In [252]:
app_train['AMT_REQ_CREDIT_BUREAU_HOUR'].value_counts()

-0.082511     264366
 12.739813      1560
 25.562138        56
 38.384462         9
-0.013093          1
               ...  
 0.020887          1
-0.029215          1
 0.013570          1
 0.029292          1
-0.009931          1
Name: AMT_REQ_CREDIT_BUREAU_HOUR, Length: 41524, dtype: int64

### AMT_REQ_CREDIT_BUREAU_DAY

In [253]:
app_train['AMT_REQ_CREDIT_BUREAU_DAY'].isnull().sum()

0

In [254]:
app_train['AMT_REQ_CREDIT_BUREAU_DAY'].value_counts()

-0.067550     264503
 9.638489       1292
 19.344528       106
 29.050566        45
 38.756605        26
               ...  
-0.018827          1
 0.002111          1
 0.009073          1
-0.024263          1
-0.008724          1
Name: AMT_REQ_CREDIT_BUREAU_DAY, Length: 41528, dtype: int64

### AMT_REQ_CREDIT_BUREAU_WEEK

In [255]:
app_train['AMT_REQ_CREDIT_BUREAU_WEEK'].isnull().sum()

0

In [256]:
app_train['AMT_REQ_CREDIT_BUREAU_WEEK'].value_counts()

-0.180649     257456
 5.071232       8208
 10.323112       199
 15.574993        58
 20.826873        34
               ...  
-0.012421          1
 0.034005          1
-0.003401          1
-0.032033          1
 0.001308          1
Name: AMT_REQ_CREDIT_BUREAU_WEEK, Length: 41528, dtype: int64

### AMT_REQ_CREDIT_BUREAU_MON

In [257]:
app_train['AMT_REQ_CREDIT_BUREAU_MON'].isnull().sum()

0

In [258]:
app_train['AMT_REQ_CREDIT_BUREAU_MON'].value_counts()

-0.310034    222233
 0.862344     33147
 2.034721      5386
 3.207099      1991
 4.379477      1076
              ...  
-0.094348         1
 0.050395         1
-0.063461         1
-0.077405         1
-0.116216         1
Name: AMT_REQ_CREDIT_BUREAU_MON, Length: 41543, dtype: int64

### AMT_REQ_CREDIT_BUREAU_QRT

In [259]:
app_train['AMT_REQ_CREDIT_BUREAU_QRT'].isnull().sum()

0

In [260]:
app_train['AMT_REQ_CREDIT_BUREAU_QRT'].value_counts()

-0.358577    215417
 0.995174     33862
 2.348925     14412
 3.702677      1717
 5.056428       476
              ...  
-0.044780         1
 0.008240         1
 0.041184         1
-0.018650         1
 0.026732         1
Name: AMT_REQ_CREDIT_BUREAU_QRT, Length: 41530, dtype: int64

### AMT_REQ_CREDIT_BUREAU_YEAR

In [261]:
app_train['AMT_REQ_CREDIT_BUREAU_YEAR'].isnull().sum()

0

In [262]:
app_train['AMT_REQ_CREDIT_BUREAU_YEAR'].value_counts()

-1.086589    71801
-0.513331    63405
 0.059928    50192
 0.633186    33628
 1.206445    20714
             ...  
 0.040465        1
-0.347133        1
 0.058601        1
-0.098512        1
 0.118037        1
Name: AMT_REQ_CREDIT_BUREAU_YEAR, Length: 41544, dtype: int64

## 特徴量の重要度

## 特徴量の評価

### 相関係数

In [263]:
corrs_columns = list(app_test.columns)

corrs = []
border = 0.01

for c in corrs_columns:
    corr = np.corrcoef(app_train[c], app_train['TARGET'])[0, 1]
    if math.isnan(corr) == False:
        if abs(corr) >= border:
            corrs.append(corr)
corrs = np.array(corrs)


corrs_sp = []
for c in corrs_columns:
    corr_sp = st.spearmanr(app_train[c], app_train['TARGET']).correlation
    if math.isnan(corr_sp) == False:
        if abs(corr_sp) >= border:
            corrs_sp.append(corr_sp)
corrs_sp = np.array(corrs_sp)

idx = np.argsort(np.abs(corrs))[::-1]
top_cols, top_importances = app_train.columns.values[idx], corrs[idx][:100]
print(top_cols, top_importances)

['OCCUPATION_TYPE' 'FLAG_EMAIL' 'FLAG_PHONE' 'NAME_INCOME_TYPE'
 'OWN_CAR_AGE' 'DAYS_ID_PUBLISH' 'AMT_INCOME_TOTAL' 'YEARS_BUILD_MODE'
 'TARGET' 'NAME_HOUSING_TYPE' 'FLAG_WORK_PHONE'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'NAME_EDUCATION_TYPE'
 'FLAG_EMP_PHONE' 'COMMONAREA_MODE' 'NAME_FAMILY_STATUS' 'APARTMENTS_MODE'
 'CNT_CHILDREN' 'NONLIVINGAPARTMENTS_AVG' 'NAME_TYPE_SUITE'
 'FLAG_CONT_MOBILE' 'AMT_CREDIT' 'BASEMENTAREA_MODE'
 'YEARS_BEGINEXPLUATATION_MODE' 'SK_ID_CURR' 'FLAG_OWN_CAR'
 'REG_REGION_NOT_LIVE_REGION' 'FLOORSMIN_AVG' 'ELEVATORS_MODE'
 'DAYS_EMPLOYED' 'EXT_SOURCE_3' 'FLAG_MOBIL' 'DAYS_REGISTRATION'
 'NONLIVINGAREA_AVG' 'WEEKDAY_APPR_PROCESS_START' 'REG_CITY_NOT_LIVE_CITY'
 'NAME_CONTRACT_TYPE' 'LIVINGAREA_AVG' 'ENTRANCES_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'EXT_SOURCE_1' 'CNT_FAM_MEMBERS'
 'CODE_GENDER' 'YEARS_BUILD_AVG' 'AMT_GOODS_PRICE'
 'REG_REGION_NOT_WORK_REGION' 'LANDAREA_MODE' 'LANDAREA_AVG'
 'REG_CITY_NOT_WORK_CITY' 'APARTMENTS_AVG' 'FLOORSMIN_MODE'
 'REGION_R

In [264]:
idx2 = np.argsort(np.abs(corrs_sp))[::-1]
top_cols2, top_importances2 = app_train.columns.values[idx][:50], corrs_sp[idx][:100]
print(top_cols2, top_importances2)

['OCCUPATION_TYPE' 'FLAG_EMAIL' 'FLAG_PHONE' 'NAME_INCOME_TYPE'
 'OWN_CAR_AGE' 'DAYS_ID_PUBLISH' 'AMT_INCOME_TOTAL' 'YEARS_BUILD_MODE'
 'TARGET' 'NAME_HOUSING_TYPE' 'FLAG_WORK_PHONE'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'NAME_EDUCATION_TYPE'
 'FLAG_EMP_PHONE' 'COMMONAREA_MODE' 'NAME_FAMILY_STATUS' 'APARTMENTS_MODE'
 'CNT_CHILDREN' 'NONLIVINGAPARTMENTS_AVG' 'NAME_TYPE_SUITE'
 'FLAG_CONT_MOBILE' 'AMT_CREDIT' 'BASEMENTAREA_MODE'
 'YEARS_BEGINEXPLUATATION_MODE' 'SK_ID_CURR' 'FLAG_OWN_CAR'
 'REG_REGION_NOT_LIVE_REGION' 'FLOORSMIN_AVG' 'ELEVATORS_MODE'
 'DAYS_EMPLOYED' 'EXT_SOURCE_3' 'FLAG_MOBIL' 'DAYS_REGISTRATION'
 'NONLIVINGAREA_AVG' 'WEEKDAY_APPR_PROCESS_START' 'REG_CITY_NOT_LIVE_CITY'
 'NAME_CONTRACT_TYPE' 'LIVINGAREA_AVG' 'ENTRANCES_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'EXT_SOURCE_1' 'CNT_FAM_MEMBERS'
 'CODE_GENDER' 'YEARS_BUILD_AVG' 'AMT_GOODS_PRICE'
 'REG_REGION_NOT_WORK_REGION' 'LANDAREA_MODE' 'LANDAREA_AVG'
 'REG_CITY_NOT_WORK_CITY'] [-0.15876168 -0.1472548  -0.14180378  0.078

### 相互情報量

In [265]:
#mi = mutual_info_classif(app_train, app_train['TARGET'])
#
#idx = np.argsort(mi)[::-1]
#mi_top_cols, mi_top_importances = app_train.columns.values[idx], corrs[idx][:100]
#
#print(mi_top_cols, mi_top_importances)

In [266]:
app_train_columns = top_cols2.tolist().copy()
#app_train_columns.append('TARGET')
app_test_columns = app_train_columns.copy()
app_test_columns.remove('TARGET')
app_test_columns

['OCCUPATION_TYPE',
 'FLAG_EMAIL',
 'FLAG_PHONE',
 'NAME_INCOME_TYPE',
 'OWN_CAR_AGE',
 'DAYS_ID_PUBLISH',
 'AMT_INCOME_TOTAL',
 'YEARS_BUILD_MODE',
 'NAME_HOUSING_TYPE',
 'FLAG_WORK_PHONE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'NAME_EDUCATION_TYPE',
 'FLAG_EMP_PHONE',
 'COMMONAREA_MODE',
 'NAME_FAMILY_STATUS',
 'APARTMENTS_MODE',
 'CNT_CHILDREN',
 'NONLIVINGAPARTMENTS_AVG',
 'NAME_TYPE_SUITE',
 'FLAG_CONT_MOBILE',
 'AMT_CREDIT',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'SK_ID_CURR',
 'FLAG_OWN_CAR',
 'REG_REGION_NOT_LIVE_REGION',
 'FLOORSMIN_AVG',
 'ELEVATORS_MODE',
 'DAYS_EMPLOYED',
 'EXT_SOURCE_3',
 'FLAG_MOBIL',
 'DAYS_REGISTRATION',
 'NONLIVINGAREA_AVG',
 'WEEKDAY_APPR_PROCESS_START',
 'REG_CITY_NOT_LIVE_CITY',
 'NAME_CONTRACT_TYPE',
 'LIVINGAREA_AVG',
 'ENTRANCES_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'EXT_SOURCE_1',
 'CNT_FAM_MEMBERS',
 'CODE_GENDER',
 'YEARS_BUILD_AVG',
 'AMT_GOODS_PRICE',
 'REG_REGION_NOT_WORK_REGION',
 'LANDAREA_MODE',
 'LANDAREA_AVG'

In [267]:
#app_train = app_train[app_train_columns].copy()
#app_test = app_test[app_test_columns].copy()
#app_train.head()

,OCCUPATION_TYPE,FLAG_EMAIL,FLAG_PHONE,NAME_INCOME_TYPE,OWN_CAR_AGE,DAYS_ID_PUBLISH,AMT_INCOME_TOTAL,YEARS_BUILD_MODE,TARGET,NAME_HOUSING_TYPE,...,YEARS_BEGINEXPLUATATION_AVG,EXT_SOURCE_1,CNT_FAM_MEMBERS,CODE_GENDER,YEARS_BUILD_AVG,AMT_GOODS_PRICE,REG_REGION_NOT_WORK_REGION,LANDAREA_MODE,LANDAREA_AVG,REG_CITY_NOT_WORK_CITY
0,0,-0.245215,1.599337,0,0.951364,0.579154,0.142129,-1.966714,1,0,...,-0.130210,-2.529986,-1.265719,0,-1.993221,-0.506611,-0.231267,-0.527166,-0.567442,-0.547236
1,1,-0.245215,1.599337,1,0.500125,1.790855,0.426792,0.670003,0,0,...,0.174145,-1.261729,-0.167636,1,0.670973,1.600798,-0.231267,-1.004839,-1.029317,-0.547236
2,0,-0.245215,1.599337,0,1.786278,0.306869,-0.427196,0.001131,0,0,...,-0.003216,0.514968,-1.265719,0,-0.002785,-1.091326,-0.231267,-0.013422,0.006817,-0.547236
3,0,-0.245215,-0.625259,0,-0.045124,0.369143,-0.142533,-0.000348,0,0,...,0.000166,0.644764,-0.167636,1,-0.002095,-0.652790,-0.231267,-0.004144,0.007004,-0.547236
4,1,-0.245215,-0.625259,0,0.245402,-0.307263,-0.199466,0.006112,0,0,...,-0.001712,0.385827,-1.265719,0,0.000476,-0.068075,-0.231267,-0.002407,-0.000517,1.827367


### Xgboostで特徴量の重要度を導出

In [268]:
data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]

y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

In [269]:
dtrain = xgb.DMatrix(X_train, label=y_train)
params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}
num_round = 50
model = xgb.train(params, dtrain, num_round)

# 重要度の上位を出力する
fscore = model.get_score(importance_type = 'total_gain')
fscore = sorted([(k, v) for k, v in fscore.items()], key=lambda tpl: tpl[1], reverse=True)
len(fscore)

[23:28:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




48

In [270]:
#test_columns = []
#for i in range(50):
#    test_columns.append(fscore[i][0])

In [271]:
#train_columns = test_columns.copy()
#train_columns.append('TARGET')

In [272]:
#app_train = app_train[train_columns].copy()
#app_test = app_test[test_columns].copy()
#app_train.head()

## CSVを出力

In [273]:
### train
app_train.to_csv(
    path_or_buf="./home-credit-default-risk/exports/app_train_fillterd.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [274]:
### test
app_test.to_csv(
    path_or_buf="./home-credit-default-risk/exports/app_test_fillterd.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

## Xgboostで学習

In [275]:
data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]

y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

In [276]:
y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)


params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

[23:29:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.71407	eval-auc:0.69928
[1]	train-auc:0.72614	eval-auc:0.70919
[2]	train-auc:0.73171	eval-auc:0.71121
[3]	train-auc:0.73573	eval-auc:0.71231
[4]	train-auc:0.73913	eval-auc:0.71308
[5]	train-auc:0.74241	eval-auc:0.71541
[6]	train-auc:0.74656	eval-auc:0.71822
[7]	train-auc:0.74914	eval-auc:0.71940
[8]	train-auc:0.75151	eval-auc:0.72133
[9]	train-auc:0.75365	eval-auc:0.72327
[10]	train-auc:0.75487	eval-auc:0.72368
[11]	train-auc:0.75690	eval-auc:0.72409
[12]	train-auc:0.75770	eval-auc:0.72395
[13]	t

[8]	train-auc:0.75247	eval-auc:0.71625
[9]	train-auc:0.75519	eval-auc:0.71755
[10]	train-auc:0.75634	eval-auc:0.71832
[11]	train-auc:0.75804	eval-auc:0.71859
[12]	train-auc:0.75942	eval-auc:0.71957
[13]	train-auc:0.76102	eval-auc:0.71988
[14]	train-auc:0.76204	eval-auc:0.72017
[15]	train-auc:0.76330	eval-auc:0.72020
[16]	train-auc:0.76457	eval-auc:0.72019
[17]	train-auc:0.76523	eval-auc:0.72086
[18]	train-auc:0.76647	eval-auc:0.72117
[19]	train-auc:0.76875	eval-auc:0.72151
[20]	train-auc:0.77011	eval-auc:0.72217
[21]	train-auc:0.77119	eval-auc:0.72227
[22]	train-auc:0.77296	eval-auc:0.72249
[23]	train-auc:0.77392	eval-auc:0.72298
[24]	train-auc:0.77493	eval-auc:0.72345
[25]	train-auc:0.77655	eval-auc:0.72337
[26]	train-auc:0.77829	eval-auc:0.72396
[27]	train-auc:0.77983	eval-auc:0.72411
[28]	train-auc:0.78113	eval-auc:0.72464
[29]	train-auc:0.78209	eval-auc:0.72488
[30]	train-auc:0.78322	eval-auc:0.72510
[31]	train-auc:0.78460	eval-auc:0.72521
[32]	train-auc:0.78592	eval-auc:0.72581
[3

In [277]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [278]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [279]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [280]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [281]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [282]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [283]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.098607,0.110636,0.135801,0.114544,0.111903
1,0.080797,0.093540,0.120069,0.089340,0.091209
2,0.025705,0.029828,0.031653,0.032207,0.027524
3,0.029422,0.029409,0.032199,0.034948,0.029036
4,0.101401,0.103915,0.092791,0.119991,0.091841


In [284]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.114298
1,100005,0.094991
2,100013,0.029383
3,100028,0.031003
4,100038,0.101988


In [285]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_all_xgb.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.022860
1,100005,0.018998
2,100013,0.005877
3,100028,0.006201
4,100038,0.020398


In [286]:
submit_all.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64